# Diff `measurements_old` and `measurements` layers

Connect to the database.

In [1]:
from estnltk.storage import PostgresStorage

storage = PostgresStorage(dbname='egcut_epi',
                          pgpass_file='~/.pgpass',
                          schema="grammarextractor",
                          role='egcut_epi_grammarextractor_create')

collection = storage.get_collection('measurements_random30000_d')

INFO:storage.py:99: connecting to host: '10.6.6.29', port: '5432', dbname: 'egcut_epi', user: 'dages'
INFO:storage.py:113: schema: 'grammarextractor', temporary: False, role: 'egcut_epi_grammarextractor_create'


In [ ]:
from estnltk.taggers import FlattenTagger

In [ ]:
flatten_tagger = FlattenTagger(input_layer='measurements',
                               output_layer='measurements_flat',
                               #output_attributes=['name', 'key', 'value', 'unit', 'DATE'],
                               output_attributes=['name', 'value', 'unit'],
                               attribute_mapping=(('REGEX_TYPE', 'name'), 
                                                  ('VALUE','value'), 
                                                  ('UNIT', 'unit')
                                                  )
                               )

collection.create_layer_buffered(tagger=flatten_tagger, overwrite=True, progressbar='notebook')

In [ ]:
flatten_tagger = FlattenTagger(input_layer='measurements5',
                               output_layer='measurements_flat5',
                               #output_attributes=['name', 'key', 'value', 'unit', 'DATE'],
                               output_attributes=['name', 'value', 'unit'],
                               attribute_mapping=(('REGEX_TYPE', 'name'), 
                                                  ('VALUE','value'), 
                                                  ('UNIT', 'unit')
                                                  )
                               )

collection.create_layer_buffered(tagger=flatten_tagger, overwrite=True, progressbar='notebook')

In [ ]:
texts = []
for idx, thing in enumerate(collection.select(layers = ['measurements', 'measurements5'])):
    texts.append(thing)

In [2]:
from collections import OrderedDict
from estnltk.taggers import DiffTagger

diff_tagger = DiffTagger(layer_a='measurements_flat',
                         layer_b='measurements_flat5',
                         output_layer='measurements_diff_1_5',
                         output_attributes=['name', 'value', 'unit']
                        )

In [3]:
meta = OrderedDict([('unchanged_annotations', 'int'),
                    ('missing_annotations', 'int'),
                    ('extra_annotations', 'int'),
                    ('unchanged_spans', 'int'),
                    ('modified_spans', 'int'),
                    ('missing_spans', 'int'),
                    ('extra_spans', 'int'),
                    ('conflicts', 'int'),
                    ('overlapped', 'int'),
                    ('prolonged', 'int'),
                    ('shortened', 'int')])

collection.create_layer(tagger=diff_tagger,
                                 overwrite=True,
                                 meta=meta,
                                 progressbar='notebook'
                                 )

INFO:collection.py:856: collection: 'measurements_random30000_d'
INFO:collection.py:877: nothing to overwrite, preparing to create a new layer: 'measurements_diff_1_5'



INFO:collection.py:942: layer created: 'measurements_diff_1_5'


## Statistics
Tabel of diff layer meta data.

In [2]:
from estnltk.taggers.standard_taggers.diff_tagger import DiffSampler


sampler = DiffSampler(collection=collection, layer='measurements_diff_1_5')

In [3]:
sampler.layer_meta.loc[:10]

,id,text_id,unchanged_annotations,missing_annotations,extra_annotations,unchanged_spans,modified_spans,missing_spans,extra_spans,conflicts,overlapped,prolonged,shortened
0,1,1,0,0,0,0,0,0,0,0,0,0,0
1,2,2,0,0,0,0,0,0,0,0,0,0,0
2,3,3,0,0,0,0,0,0,0,0,0,0,0
3,4,4,0,0,0,0,0,0,0,0,0,0,0
4,5,5,0,0,0,0,0,0,0,0,0,0,0
5,6,6,0,0,0,0,0,0,0,0,0,0,0
6,7,7,0,0,0,0,0,0,0,0,0,0,0
7,8,8,0,0,0,0,0,0,0,0,0,0,0
8,9,9,0,0,0,0,0,0,0,0,0,0,0
9,10,10,0,0,0,0,0,0,0,0,0,0,0


In [4]:
sampler.layer_meta.sum()

id                       449565105
text_id                  449565105
unchanged_annotations        13326
missing_annotations            528
extra_annotations             1140
unchanged_spans              13326
modified_spans                 101
missing_spans                  427
extra_spans                   1039
conflicts                      388
overlapped                       0
prolonged                      388
shortened                        0
dtype: int64

## Random samples
The following iterations give different kind of random samlples without replacement. The domain names of samples (`'modified_spans'`, `missing_spans'` etc) correspond to the summary table columns.

In [9]:
list(sampler.sample_spans(k=10, domain='conflicts'))[0]

ProgrammingError: can't adapt type 'numpy.int64'

In [7]:
for text_id, span_index, span in sampler.sample_spans(k=10, domain='conflicts'):
    print('text_id: {}, span index: {}'.format(text_id, span_index))
    span.display()

ProgrammingError: can't adapt type 'numpy.int64'

In [ ]:
texts = []
for idx, thing in enumerate(collection.select(layers = ['measurements_old_new', 'measurements_flat3'])):
    texts.append(thing)

In [ ]:
texts[16101][1].measurements_old_new[0]

In [ ]:
c = 0
for text_id, span_index, spans in sampler.sample_spans(50000, 'conflicts'):
    if spans[0].value == spans[1].value and spans[0].name == spans[1].name:
        continue
    if spans[0].name[0].strip() == 'VERERÕHK' and spans[1].name[0].strip() == 'VERERÕHK':
        continue
    if spans[0].name[0].strip() == 'PULSS' and spans[1].name[0].strip() == 'PULSS':
        continue    
    if spans[0].name != spans[1].name:# and spans[0].value == spans[1].value:
        continue
    print('text_id: {}, span index: {}'.format(text_id, span_index))
    if c > 150:
        break
    c += 1
    spans[0].display()
    spans[1].display()

In [ ]:
c

In [ ]:
c

In [ ]:
for text_id, span_index, spans in sampler.sample_spans(2, 'overlapped'):
    print('text_id: {}, span index: {}'.format(text_id, span_index))
    spans[0].display()
    spans[1].display()

In [ ]:
for text_id, span_index, spans in sampler.sample_spans(2, 'prolonged'):
    print('text_id: {}, span index: {}'.format(text_id, span_index))
    spans[0].display()
    spans[1].display()

In [ ]:
for text_id, span_index, spans in sampler.sample_spans(2, 'shortened'):
    print('text_id: {}, span index: {}'.format(text_id, span_index))
    spans[0].display()
    spans[1].display()

Display `measurements_diff_1_2` layer.

In [ ]:
from estnltk import Layer
Layer.print_start_end = True

texts = collection.select(layers=['measurement_tokens',
                                  'measurements_old',
                                  'measurements_flat',
                                  'measurements_diff_old_flat'],
                          keys=[4])
text_id, text = next(texts)
text.measurements_diff_old_flat

In [ ]:
storage.close()